In [1]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

## Load the training data into feature matrix, class labels, and event ids:

In [15]:
from proj1_helpers import *
DATA_TRAIN_PATH = "../data/train.csv" # TODO: download train data and supply path here 
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)

In [21]:
with open(DATA_TRAIN_PATH, "r") as f:
            title_list = f.readline().split(",")

jet_num_col = title_list.index("PRI_jet_num")

category_tx = np.genfromtxt(
    DATA_TRAIN_PATH, dtype=str, delimiter=",", skip_header=1, usecols=[jet_num_col])
category_tx


array(['2', '1', '1', ..., '1', '0', '0'], dtype='<U1')

In [49]:
class Dataset:
    def __init__(self, data_pth, data_type):
        self.data_pth = f"{data_pth}/{data_type}.csv"
        self.data_type = data_type

        self.data = []
        self.labels = []
        self.ids = []

        self.col_names = self.read_col_names()
        self.num_cols = len(self.col_names)

    def load_data(self):
        """Load the data from the csv file."""

        y, tX, ids = load_csv_data(self.data_pth)
        self.ids = ids
        self.labels = y
        self.data = tX

    def read_col_names(self):
        """Read the column names from the csv file."""

        with open(self.data_pth, "r") as f:
            col_names = f.readline().strip().split(",")
        return col_names
    
    def data_imputation(self, method="mean"):
        """Impute the missing values in the data."""

        for col in range(self.num_cols):
            col_data = self.data[:, col]
            if method == "mean":
                col_data[col_data == -999.0] = np.nanmean(col_data[col_data != -999.0])
                col_data[np.isnan(col_data)] = np.nanmean(col_data)
            elif method == "median":
                col_data[col_data == -999.0] = np.nanmedian(col_data[col_data != -999.0])
                col_data[np.isnan(col_data)] = np.nanmedian(col_data)
            else:
                col_data[col_data == -999.0] = 0.0
                col_data[np.isnan(col_data)] = 0.0

        if method == "mean":
            self.data = self.mean_imputation()
    
    def data_normalization(self):
        """Normalize the data, zero-mean and standardization."""

        mean_data = np.mean(self.data, axis=0)
        self.data = self.data - mean_data
        std_data = std_data / np.std(self.data, axis=0)


In [36]:
train_dataset = Dataset("../data", "train")
train_dataset.load_data()

In [45]:
id_col = train_dataset.col_names.index("Id")
label_col = train_dataset.col_names.index("Prediction")
jet_num_col = train_dataset.col_names.index("PRI_jet_num")
special_col = [id_col, label_col, jet_num_col]
float_col_ids = []
float_col_names = []
category_col_names = ['jetnum3', 'jetnum2', 'jetnum1', 'jetnum0']
for idx in range(len(title_list)):
    if idx not in special_col:
        float_col_ids.append(idx)
        float_col_names.append(title_list[idx])
        full_col_names = ['x_bias'] + float_col_names + category_col_names
len(full_col_names)

34

### Normalize Data

In [16]:
def normalize(x):
    """Normalize the dataset x."""
    mean_x = np.mean(x, axis=0)
    std_x = np.std(x, axis=0)
    x = (x - mean_x) / std_x
    return x

tX_norm = normalize(tX)

## Do your thing crazy machine learning thing here :) ...

## Generate predictions and save ouput in csv format for submission:

In [4]:
DATA_TEST_PATH = "../data/test.csv" # TODO: download train data and supply path here 
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)

In [31]:
OUTPUT_PATH = '' # TODO: fill in desired name of output file for submission
y_pred = predict_labels(weights, tX_test)
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)